# **Análisis Estadístico de Redes Sociales: Taller 4**

## Valentina Cardona Saldaña

## **WEB SCRAPING: RECOLECCIÓN DE TRANSCRIPCIONES**

### **LIBRERÍAS**

In [1]:
# Paquetes
import pandas as pd
import numpy as np
import time

import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

### **FUNCIONES**

In [2]:
# Función para extracción de links
def WS_LINKS():

    títulos = []
    links = []

    # Función para recolectar links
    def Encontrar_links():
        try:
            # Encontrar bloque de playlist
            block = driver.find_element(By.CLASS_NAME, "style-scope ytd-playlist-video-list-renderer")
            search_list = block.find_elements(By.XPATH, "//h3[contains(@class, 'ytd-playlist-video-renderer')]")

            # Recolectar links y títulos
            for search in search_list:
                all_links = search.find_elements(By.TAG_NAME, "a")
                for a in all_links:
                    if str(a.get_attribute("href")).startswith("https://www.youtube.com/") and a.get_attribute("href") not in links: 
                        títulos.append(a.text)
                        links.append(a.get_attribute("href"))
                    else:
                        pass

                # Desplazar hacia abajo para ver todos los títulos de los vídeos
                driver.execute_script("arguments[0].scrollIntoView();", search)
        except:
            pass

    print("Links están siendo recolectados")
    Encontrar_links()
    
    # Crear variable global
    global búsquedas
    
    # Crear dataframe y exportar
    búsquedas = pd.DataFrame(zip(títulos, links), columns = ['Títulos', 'Links'])
    búsquedas.to_csv('./Datos/Caso03_links.csv', index = False)
    
    # Imprimir resultados
    print(búsquedas.shape[0])
    return búsquedas

In [3]:
# Función para extracción de contenido
def WS_CONTENT(datos):
    # Crear listas vacías para almacenar información
    ID = []
    Títulos = []
    Transcripción = []
    
    # Visite cada link uno por uno para hacer scraping a la información
    print('Visitar los links y recopilar información acaba de comenzar.')

    for i in range(len(datos)):
        # Abrir link
        try:
            # Ingresar a la página
            driver.get(datos['Links'][i])
            ID.append(datos['ID'][i])
            Títulos.append(datos['Títulos'][i])
            time.sleep(7)
            # Pausar el vídeo
            driver.find_element(By.CLASS_NAME, "ytp-play-button").click()
            time.sleep(3)
            # Dar clic en ..más
            driver.find_element(By.XPATH, "//tp-yt-paper-button[@id='expand']").click()
            time.sleep(2)
            # Dar clic en Mostrar transcripción
            driver.find_element(By.XPATH, "//button[contains(@aria-label, 'Mostrar transcripción')]").click()
            time.sleep(7)
            # Recoger transcripciones
            block = driver.find_element(By.ID, "segments-container")
            segmentos = block.find_elements(By.XPATH, "//yt-formatted-string[contains(@class, 'segment-text style-scope ytd-transcript-segment-renderer')]")
            textos = []
            for segmento in segmentos:
                textos.append(segmento.text)
            
            Transcripción.append(' '.join(textos))
            
        except NoSuchElementException:
            print(f'No se pudo realizar scraping en el ID {datos['ID'][i]}')
            pass

    # Crear variable global
    global Caso03_Transcripciones
        
    # Crear dataframe
    Caso03_Transcripciones = pd.DataFrame(zip(ID, Títulos, Transcripción), columns = ['ID', "Títulos", 'Transcripciones'])

    # Exportar
    Caso03_Transcripciones.to_csv('./Datos/Caso03_Transcripciones.csv', index = False)
        
    # Imprimir resultados
    print(f'Scraping de los links ha resultado en {Caso03_Transcripciones.shape[0]} filas de {len(datos)}')

### **WEB DRIVER**

In [4]:
# Web driver
## Usar Brave chromedriver

# Windows
browser = "C:\\Program Files\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"
# MacOS
#browser = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'

options = webdriver.ChromeOptions()
options.binary_location = browser
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options = options)
## Esperar a que cargue

### **EXTRACCIÓN DE LINKS**

In [5]:
# Abrir la página de la playlist
driver.get("https://www.youtube.com/playlist?list=PLbtegW3d3L4Id_lrAgBRDGf-k_bwEYH6_")
## Esperar a que cargue
## Asegurarse de ir hasta el final de la página y regresar al inicio para que pueda recoger todos los títulos

In [ ]:
# Extracción links
WS_LINKS()

### **EXTRACCIÓN DE CONTENIDO**

In [7]:
# Importar datos extraídos anteriormente
datos = pd.read_csv("./Datos/Caso03_links.csv")

# Generar ids
datos['ID'] = datos.index + 1

# Reorganizar las columnas
datos = datos[["ID", "Títulos", "Links"]]

datos

,ID,Títulos,Links
0,1,Caso 03 | Edwin Leonardo Toro | Subcaso Casana...,https://www.youtube.com/watch?v=Dlfn1RekIeA&li...
1,2,Invitación Caso 04 | Juan Manuel Grajales y Ge...,https://www.youtube.com/watch?v=KnwzIgiijTA&li...
2,3,C 03 - 04 | Sargento Primero (r) Jaime Coral y...,https://www.youtube.com/watch?v=pkgwJ2y4eyU&li...
3,4,Casos 03 - 04 | Soldado Profesional (r) Gabrie...,https://www.youtube.com/watch?v=yPaWsnpeWdk&li...
4,5,Casos 03 - 04 | Segundo Sargento (r) Armando F...,https://www.youtube.com/watch?v=9rXtyCdwLHI&li...
...,...,...,...
433,434,Caso 03 | Henry Hernán Acosta | Subcaso Casana...,https://www.youtube.com/watch?v=VOFlxqzJqLw&li...
434,435,Caso 03 | Jorge Arturo Romero Romero | Subcaso...,https://www.youtube.com/watch?v=NmU4Q32oLMM&li...
435,436,Caso 03 | Guillermo Quiñónez | Subcaso Casanar...,https://www.youtube.com/watch?v=vjzz91wkNHo&li...
436,437,Caso 03 | Leandro Eliécer Moná Cano | Subcaso ...,https://www.youtube.com/watch?v=HS3oQWBvje8&li...


In [8]:
# Extraer contenidos
WS_CONTENT(datos)

Visitar los links y recopilar información acaba de comenzar.
Scraping de los links ha resultado en 438 filas de 438


In [9]:
Caso03_Transcripciones

,ID,Títulos,Transcripciones
0,1,Caso 03 | Edwin Leonardo Toro | Subcaso Casana...,Catalina Díaz Gómez yo soy magistrada de la sa...
1,2,Invitación Caso 04 | Juan Manuel Grajales y Ge...,bueno eh nuevamente a todas a todos a los magi...
2,3,C 03 - 04 | Sargento Primero (r) Jaime Coral y...,al al al al al bien Buenos días a todos a toda...
3,4,Casos 03 - 04 | Soldado Profesional (r) Gabrie...,Muy buenos días mi nombre es Hugo Escobar Fern...
4,5,Casos 03 - 04 | Segundo Sargento (r) Armando F...,muy buenos días hoy lunes 2 de agosto del 2021...
...,...,...,...
433,434,Caso 03 | Henry Hernán Acosta | Subcaso Casana...,Muy buenos días a todas y todos Mi nombre es Ó...
434,435,Caso 03 | Jorge Arturo Romero Romero | Subcaso...,Muy buenos días mi nombre es Óscar Parra Vera ...
435,436,Caso 03 | Guillermo Quiñónez | Subcaso Casanar...,Muy buenos días mi nombre es Óscar Parra Vera ...
436,437,Caso 03 | Leandro Eliécer Moná Cano | Subcaso ...,Muy buenos días mi nombre es Juan Pablo Cardon...


In [10]:
driver.quit()